### python -m pip install pythonnet

In [87]:
from pathlib import Path
from enum import Enum
import json

In [88]:
data_folder = Path('./test_data/')
material_data = data_folder / 'material_data.json'
column_data = data_folder / 'column_data.json'
cross_section_data = data_folder / 'cross_section_data.json'
analytical_levels = data_folder / 'analytical_levels.json'


#start ETABS application.
myETABSObject.ApplicationStart()

#close the program.
ret = myETABSObject.ApplicationExit(False)


In [89]:
import clr
clr.AddReference("System.Runtime.InteropServices")
from System.Runtime.InteropServices import Marshal

#set the following path to the installed ETABS program directory
clr.AddReference(R'C:\Program Files\Computers and Structures\ETABS 22\ETABSv1.dll')
import ETABSv1 as etabs

#create API helper object
helper = etabs.cHelper(etabs.Helper())

try:
    myETABSObject = etabs.cOAPI(helper.GetObject("CSI.ETABS.API.ETABSObject"))
except:
    print("No running instance of the program found or failed to attach.")

#create SapModel object
SapModel = etabs.cSapModel(myETABSObject.SapModel)

N_mm_C = etabs.eUnits.N_mm_C

display(myETABSObject)
display(SapModel)


In [90]:
with open(analytical_levels, 'r') as file:
    levels = json.load(file)
level_dict = {k:v for k,v in levels}
sorted_level_dict = {k: v for k, v in sorted(level_dict.items(), key=lambda item: item[1])}
story_names = list(sorted_level_dict.keys())[1:]
story_heights = []
base_level = 0
for i,level in enumerate(sorted_level_dict.values()):
    if i == 0:
        base_level = level
        previous_level = level
        continue
    story_height = level - previous_level
    previous_level = level
    story_heights.append(story_height)

story_names
story_heights

def create_stories(SapModel,base_level:float,story_names:list[str],story_heights:list[float],unit = N_mm_C) -> tuple[int,list[str],list[float]]:
    ret = SapModel.SetPresentUnits(unit)
    stories_count = len(story_names)
    IsMasterStory = [False] * stories_count
    SimilarToStory = [''] * stories_count
    SpliceAbove = [False] * stories_count
    SpliceHeight = [0.0] * stories_count
    color = [0] * stories_count
    [ret, story_names, story_heights, IsMasterStory, SimilarToStory, SpliceAbove, SpliceHeight, color] = SapModel.Story.SetStories_2(
        base_level,
        stories_count,
        story_names,
        story_heights,
        IsMasterStory,
        SimilarToStory,
        SpliceAbove,
        SpliceHeight,
        color
    )
    return (ret, list(story_names), list(story_heights))

ret,_,_ = create_stories(SapModel,base_level,story_names,story_heights)


In [91]:
SapModel = None
myETABSObject = None